# **Classification of Gene Sequences Into Their Gene Families**

First, let's download the data for this notebook.

The following commands will download (`wget`) the file `gene-family-classification.tar.gz` and then unarchive and decompress it (`tar`), resulting in the folder `gene-family-classification/`.
Then, the folder will be renamed (`mv`) to `data/`, and the now superfluous file `gene-family-classification.tar.gz` will be removed (`rm`).

In [ ]:
! wget http://www.tnt.uni-hannover.de/edu/vorlesungen/AMLG/data/gene-family-classification.tar.gz
! tar -xzvf gene-family-classification.tar.gz
! mv -v gene-family-classification/ data/
! rm -v gene-family-classification.tar.gz

Second, let's group some imports here.

In [ ]:
import numpy as np
import numpy.typing as npt
import re
from typing import Any, List

# Biological sequence representation for machine learning

Machine learning models generally require input to be numerical values organized in the form of feature matrices.
Biological sequence data is however usually provided in a string-like format.

There are three basic approaches to represent (or in other words, encode) biological sequence data:

1. Ordinal encoding
2. One-hot encoding
3. Biological sequences as a "language" through _k_-mer counting

Before we start, however, we need a function that harmonizes biological sequence strings and converts them into more convenient `numpy` arrays:

##### ❓ **Q1** &mdash; Converting biological sequence strings to `numpy` arrays

Complete the following function to harmonize (lowercase to uppercase, use `ACGTN` only) the biological sequence strings.

In [ ]:
def sequence_to_array(sequence: str) -> npt.NDArray[Any]:
    # Harmonize lowercase and uppercase characters
    # YOUR CODE

    # Replace all characters in sequence that are not 'A', 'C', 'G', or 'T'
    # with 'N'
    # YOUR CODE

    # Format our output array
    array = np.array(list(sequence))
    array = np.reshape(array, newshape=(len(array), 1))

    return array

In [ ]:
print(sequence_to_array(sequence="GATTACA").T)

## Ordinal encoding

In this approach, we encode bases (i.e., categorical features) as ordinal integers, i.e., integers having a relative ordering.

In [ ]:
from sklearn.preprocessing import OrdinalEncoder


ordinal_encoder = OrdinalEncoder()
ordinal_encoder.fit(X=[["A"], ["C"], ["G"], ["T"], ["N"]])

array = sequence_to_array(sequence="GATTACAN")
encoded_sequence = ordinal_encoder.transform(X=array)

with np.printoptions(
    formatter={"float_kind": lambda x: f"{x:3.0f}", "str_kind": lambda x: f"  {x}"}
):
    print(f"Sequence:         {array.T}")
    print(f"Encoded sequence: {encoded_sequence.T}")

## One-hot encoding

Another approach is to use one-hot encoding to represent the biological sequence.
This is widely used in deep learning methods.
Each symbol gets transformed into a one-hot encoding vector.
A one-hot encoding vector is a vector in which all elements are zero except for one element which is set to one.
The position of the one in the vector encodes the symbol.
For further processing, these one-hot encoded vectors can for example be concatenated.

##### ❓ **Q2** &mdash; One-hot encoding for biological sequences

Similar to the ordinal encoding, implement and perform the one-hot encoding for the sequence `GATTACA`.
Use the `OneHotEncoder` class from `sklearn.preprocessing`.
When instantiating an encoder from the class, use `sparse_output=False`.

In [ ]:
# YOUR CODE

with np.printoptions(
    formatter={"float_kind": lambda x: f"{x:3.0f}", "str_kind": lambda x: f"  {x}"}
):
    print(f"{array.T}")
    print(f"{encoded_sequence.T}")

## Biological sequences as a "language" through _k_-mer counting

A hurdle that still remains is that none of these above methods results in arrays of uniform length, which is often a necessity.
So with the above methods, you have to resort to things like truncating sequences or padding with "N" to get arrays of uniform length.

Luckily there is the concept of _k_-mers.

_k_-mers are substrings of length $k$ contained within a biological sequence.
They are a fairly simple concept that turns out to be tremendously powerful.

Usually, the term _k_-mer refers to all of a sequence's subsequences of length $k$.
Using this definition, the sequence `ACGT` would have four monomers (`A`, `C`, `G`, `T`), three 2-mers (`AC`, `CG`, `GT`), two 3-mers (`ACG`, `CGT`), and one 4-mer (`ACGT`).
More generally, a sequence of length $L$ has $L - k + 1$ _k_-mers and $n^k$ total possible _k_-mers, where $n$ is the number of possible monomers.

In some sense, by using _k_-mers, we can also incorporate the concepts of "words" (_k_-mers) and "sentences" (sequences of _k_-mers) from natural language processing.

The following function `build_kmers()` returns a list of _k_-mer "words" for a given sequence.

In [ ]:
def build_kmers(sequence: str, k: int) -> List[str]:
    return [sequence[i : i + k] for i in range(len(sequence) - k + 1)]

We can use this function to generate a language-like sentence from a biological sequence:

In [ ]:
sequence = "GTGCCCAGGTTCAGTGA"
words = build_kmers(sequence=sequence, k=3)
sentence = " ".join(words)

print(f"Sequence : {sequence}")
print(f"Words    : {words}")
print(f"Sentence : {sentence}")

In a next step, we may use a **Bag of Words** (BoW) model to represent our "sentence".

> A Bag of Words (BoW) model is a popular and simple way to represent text data in natural language processing (NLP) and information retrieval.
> In this model, a document is represented as an unordered set of words, disregarding grammar and word order but keeping track of word frequency.
> The name "Bag of Words" implies that the model is only concerned with the occurrence of words in a document, not with their order or structure.

> Here's how the Bag of Words model typically works:
>
> 1. Tokenization: The first step is to break down a text into individual words or tokens. This process involves removing punctuation, converting all words to lowercase, and splitting the text into individual words.
> 2. Vocabulary Building: Create a vocabulary of all unique words in the entire corpus (collection of documents). Each unique word in this vocabulary is assigned a unique index.
> 3. Vectorization: Represent each document as a vector in the space of the vocabulary. The vector has the same length as the vocabulary, and each element of the vector corresponds to the frequency of a word from the vocabulary in the document.

> Here's a simple example:
>
> Consider two documents:
>
> - Document 1: "The cat in the hat."
> - Document 2: "The quick brown fox."
> - The vocabulary is: {the, cat, in, hat, quick, brown, fox}.
>
> The Bag of Words representation for these documents would be:
>
> - Document 1: [2, 1, 1, 1, 0, 0, 0]
> - Document 2: [1, 0, 1, 0, 1, 1, 1]

##### ❓ **Q3** &mdash; Employing the BoW model in the context of biological sequence data

The BoW model can&nbsp;&mdash;&nbsp;analogous to its use in NLP&nbsp;&mdash;&nbsp;be employed for biological sequence data.

1. Use the function `build_kmers()` to build a word corpus from the given sequence corpus. Use $k=3$.
2. Use the `CountVectorizer` from `sklearn.feature_extraction.text` to fit the BoW model to the word corpus.
3. Vectorize new "documents" using the trained BoW model.

> Use the function `toarray()` to convert sparse matrix objects returned by `CountVectorizer`s `transform()` and `fit_transform()` functions into dense representation as a NumPy array.

In [ ]:
sequence_corpus = ["GATTCCAG", "GATTACA", "NAAGT"]

In [ ]:
# Build the word corpus

# YOUR CODE

In [ ]:
# Fit the BoW model to the word corpus

from sklearn.feature_extraction.text import CountVectorizer

vectorizer = # YOUR CODE
word_vectorization = # YOUR CODE

print(f"Word vectorization: {word_vectorization.toarray()}")
print(f"Feature names: {vectorizer.get_feature_names_out()}")

In [ ]:
# Vectorize new documents using the trained BoW model

# YOUR CODE

# A real-world end-to-end exercise

In this end-to-end exercise, the goal is to classify gene sequences into their gene families.

We provide gene sequences from three species in the `data/` folder:

- [Chimpanzee](https://en.wikipedia.org/wiki/Chimpanzee) (_Pan troglodytes_): `chimpanzee.txt`
- [Dog](https://en.wikipedia.org/wiki/Dog) (_Canis familiaris_): `dog.txt`
- [Human](https://en.wikipedia.org/wiki/Human) (_Homo sapiens_): `human.txt`

Each file contains two tab-separated columns.
The first column (column header: `sequence`) contains the relevant gene sequence.
The second column (column header: `class`) contains the corresponding gene family, coded as integer:

| Identifier | Gene family                         |
|------------|-------------------------------------|
|          0 | G protein-coupled receptors (GPCRs) |
|          1 | Tyrosine kinase                     |
|          2 | Tyrosine phosphatase                |
|          3 | Synthetase                          |
|          4 | Synthase                            |
|          5 | Ion channel                         |
|          6 | Transcription factor                |

##### ❓ **Q4** &mdash; Reading data in with `pandas`

Use `pandas`' [`read_table()`](https://pandas.pydata.org/docs/reference/api/pandas.read_table.html) function to read in the three files.

In [ ]:
import pandas as pd

chimp_dna = # YOUR CODE
dog_dna = # YOUR CODE
human_dna = # YOUR CODE

In [ ]:
human_dna.head()

##### ❓ **Q5** &mdash; Building _k_-mer "words" for the data

Convert the sequences into short overlapping _k_-mer "words" of length $k=6$.
For now, do this only for the human data.
Add the _k_-mer words as additional column to the `human_dna` dataframe.

In [ ]:
# YOUR CODE

##### ❓ **Q6** &mdash; Building "sentences"

For the human data, convert the lists of _k_-mers into sentences of words (separated by one whitespace) that can be used to fit the BoW model.

In [ ]:
human_corpus = # YOUR CODE

##### ❓ **Q7** &mdash; Fit a BoW model

Use the `CountVectorizer` from `sklearn.feature_extraction.text` to fit a BoW model to the `human_corpus`.
Also, vectorize the entire corpus.

> Play around with the `ngram_range` argument of `CountVectorizer()` to introduce more context into the vectorizations.

In [ ]:
vectorizer = # YOUR CODE
human_vectorizations = # YOUR CODE
print(f"Vectorizations shape: {human_vectorizations.shape}")

##### ❓ **Q8** &mdash; Split the data into a training and a test set

Extract the class labels from the `human_dna` dataframe.
Then, use the function [`train_test_split()`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) from `sklearn.model_selection` to split the vectorizations _and_ the class labels into a training set (80%) and a test set (20%).

In [ ]:
# YOUR CODE

##### ❓ **Q9** &mdash; Train a classifier

Take a classifier of your choice to classify the data.

> Try for example:
>
> - `sklearn.naive_bayes.MultinomialNB`
> - `sklearn.tree.DecisionTreeClassifier`
> - `sklearn.ensemble.RandomForestClassifier`

In [ ]:
# YOUR CODE

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay

ConfusionMatrixDisplay.from_predictions(y_true=y_test, y_pred=y_pred)

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

print(f"Accuracy  : {accuracy_score(y_true=y_test, y_pred=y_pred):.4f}")
print(
    f"Precision : {precision_score(y_true=y_test, y_pred=y_pred, average='weighted'):.4f}"
)
print(
    f"Recall    : {recall_score(y_true=y_test, y_pred=y_pred, average='weighted'):.4f}"
)
print(f"F1 score  : {f1_score(y_true=y_test, y_pred=y_pred, average='weighted'):.4f}")

# Enhancements

- Try to use different `max_depth`s for the `DecisionTreeClassifier`.
- The classes are imbalanced. Try to upsample the minority classes using `resample()` from `sklearn.utils`. Also try to downsample the majority class(es).
- It could be beneficial to tune some hyperparameters (e.g., `CountVectorizer`'s `ngram_range` parameter and the _k_-mer length).